Import modules

In [ ]:
from gtfs_realtime_utils import *
from utils import *
import pandas as pd
pd.options.display.float_format = '{:.1f}'.format
ROOT = Path("../")
ROOT.resolve()

Set global parameters

In [2]:
date = 20240923
date_with_dashes = make_date_with_dashes(date)
region = 'yorkshire'

Read the pre-processed real time data for the set date and region

In [ ]:
day_data = pd.read_csv(ROOT / f"data/gtfs-rt/csv/{date}.csv", low_memory=False)
try:
    next_day_data = pd.read_csv(ROOT / f"data/gtfs-rt/csv/{date+1}.csv", low_memory=False)
    rt_data = pd.concat([day_data, next_day_data])
except:
    print('No next day of data to combine.')
    rt_data = day_data

Add a human-readable time column to sense check things.

In [ ]:
rt_data['human_datetime'] = pd.to_datetime(rt_data['timestamp'], unit='s')
rt_data.head()

We need to fill in missing trip_ids using the following logic:
- start at row 0. we need something to track which row we're on. lets say i.
- if it has a trip id, go to the next row. keep a note of this row number, lets say k.
- if the next row doesn't have a trip_id, go to the next row.
- if the next row does have a trip id, compare that to the previou trip_id.
- if they're the same, fill in all the previous ones with that trip_id.
- if they're not, don't do anything.

Now, we take the list of trip_ids, apply the function to fill the gaps, then re-apply the list of filled in trip_ids as the 'trip_id' column in our dataframe.

In [ ]:
list_of_trip_ids = rt_data.trip_id.to_list()
filled_gaps_trip_ids = fill_trip_ids(list_of_trip_ids)
print(fraction_with_trip_id(list_of_trip_ids))
print(fraction_with_trip_id(filled_gaps_trip_ids))
rt_data['trip_id'] = filled_gaps_trip_ids

Do the same as above for start dates

In [ ]:
list_of_start_dates = rt_data.start_date.to_list()
filled_gaps_start_dates = fill_trip_ids(list_of_start_dates)
rt_data['start_date'] = filled_gaps_start_dates

Ensure we are only considering services that started on the current date.

In [6]:
rt_data = rt_data[rt_data.start_date == date]

Ideally we would keep all the data that has location info, but some of these still have no trip_id. There is no clear way to know which trip they are for. So, we will remove them from the data. These are always at the start/end of routes and so won't signifcantly affect our real time journey matching. In any case, we will fill in missing stops in the sequence using the timetable. So in the end, the lost data will not be significant.

In [ ]:
def remove_missing_trip_ids(df):
    result = df[(df.stop_sequence != 0) & (df.trip_id != '')]
    return result
clean_rt_data = remove_missing_trip_ids(rt_data)
print('Percentage of location data usable:', round(len(clean_rt_data)*100/len(rt_data), 3))

Now need to ensure there is only 1 timestamp per trip, per stop. 

Priority order is:
1) `stopped at` - a bus is currently stopped at the stop
2) `incoming at` - a bus is about to stop at the stop.
3) `in transit to` - a bus is on its way to the stop.

Data is ordered by timestamp ascending downwards, so "last" is most recent and first is earliest.

In [ ]:
# a status of 0 means the bus is incoming at the stop. Remove duplictaes, 
# taking the last (closest to stop) only.
stopped_at = clean_rt_data.loc[clean_rt_data.status == 0].copy()
stopped_at.drop_duplicates(subset=['trip_id', 'stop_sequence'], keep='last', inplace=True)

In [9]:
# a status of 1 means the bus is stopped at the stop. Remove duplictaes, 
# taking the first (earliest time the bus was stopped at the stop) only.
incoming_at = clean_rt_data.loc[clean_rt_data.status == 1].copy()
incoming_at.drop_duplicates(subset=['trip_id', 'stop_sequence'], keep='first', inplace=True)

In [10]:
# a status of 2 means the bus is in transit to the stop. Remove duplictaes, taking the last 
# (most recent time it was in transit to a stop) (closest to stop) only.
in_transit = clean_rt_data.loc[clean_rt_data.status == 2].copy()
in_transit.drop_duplicates(subset=['trip_id', 'stop_sequence'], keep='last', inplace=True)

Re-combine this data and re-sort.

In [11]:
combined = pd.concat([stopped_at, incoming_at, in_transit])
combined.sort_values(by=['trip_id', 'stop_sequence', 'timestamp'], inplace=True)

Now we pick one timestamp per trip based on the priority given above.

In [12]:
# Define a custom sort key
def prioritise_stop_status(data):
    priority_mapping = {"status": {1: 0, 0: 1, 2: 2}}
    data["priority"] = data["status"].map(priority_mapping["status"])

    # Group by and select the row with the highest priority
    result = data.sort_values(by=["trip_id", "stop_sequence", "priority"]) \
            .groupby(["trip_id", "stop_sequence"]) \
            .first().reset_index()

    # Drop the helper column
    result = result.drop(columns=["priority"])
    result.sort_values(by=['trip_id', 'stop_sequence', 'timestamp'], inplace=True)
    return result

combined_prioritised = prioritise_stop_status(combined)

Now we need to load the original timetable. We'll use the `stop_times.txt` file to compare with our live location data. The other files we will use a bit further down to create the "real" GTFS timetable.

In [ ]:
# Define the directory where the timetable files are
EXTRACT_DIR = ROOT / f"18SepGB_GTFS_Timetables_Downloaded/{region}"

# Load the various timetable files into pandas dataframes.
agencies, routes, trips, stops, stop_times, calendar, \
    calendar_dates, feed_info, shapes = load_full_gtfs(EXTRACT_DIR, include=['feed_info.txt', 'shapes.txt'])

In [14]:
# This join shows us where we have gaps in the timetable and lets us work out which ones we can interpolate.
complete_stop_time_data = stop_times.merge(combined_prioritised, on=['trip_id', 'stop_sequence'], how='left')
simplified_stop_time_data = complete_stop_time_data.loc[:, \
    ['trip_id', 'arrival_time', 'departure_time', 'stop_sequence', 'timestamp', 'human_datetime']]

group by 'trip_id' and keep rows that have at least 1 value in the "timestamp" column

In [15]:
# Group by 'trip_id' and filter groups with at least one non-null 'timestamp'
complete_stop_time_data = complete_stop_time_data.groupby('trip_id').filter(lambda group: group['timestamp'].notna().any())

In [16]:
# Convert times to UNIX timestamps
complete_stop_time_data['unix_arrival_time'] = complete_stop_time_data['arrival_time'].apply(\
    lambda x: gtfs_time_to_unix_timestamp(x, date_with_dashes))

complete_stop_time_data['unix_departure_time'] = complete_stop_time_data['departure_time'].apply(\
    lambda x: gtfs_time_to_unix_timestamp(x, date_with_dashes))

In [17]:
def interpolate_missing_times(row):
        r_n = row['timestamp']
        t_n = row['unix_arrival_time']
        # @TODO add interpolation for departure time as well. Can speed up by checking if the're
        #  equivalent first and then doing calculation if needed.
        r_pre = row['r_pre']
        r_next = row['r_next']
        t_pre = row['t_pre']
        t_next = row['t_next']
        # middle of values
        # if current timestamp doesnt exist, but next and previous values do.
        if pd.isna(r_n):
            if not pd.isna(t_n) and not pd.isna(r_pre) and not pd.isna(r_next) and not pd.isna(t_pre) and not pd.isna(t_next):
                try:
                    interpolated_time = round(r_pre + (((t_n - t_pre)/(t_next - t_pre)) * (r_next - r_pre)), 0)
                except:
                    #  If the above fails e.g. in the case of VJb3e8a46add17e4a54287ae84eaf808488812ddaa Yorkshire, due to timetabling error.
                     interpolated_time = np.nan
                return interpolated_time
            
            ### REMOVING THIS FOR NOW - we don't want to extrapolate values.

            # # deal with first missing values. (due to no start real time)
            # # We use the next time to work backwards
            # if pd.isna(r_pre) and not pd.isna(r_next) and not pd.isna(t_pre) and not pd.isna(t_next):
            #      interpolated_time = round(t_pre + ((r_next - t_pre) / (t_next - t_pre)), 0)
            #      return interpolated_time
            
            # # deal with end stops (due to no end real time appearing)
            # # We use the latest available time to fill forwards.
            # if pd.isna(r_next) and not pd.isna(t_next) and not pd.isna(t_pre):
            #     interpolated_time = round(t_n + (r_pre - t_pre), 0)
            #     return interpolated_time
            
        else:
            return r_n
        
def fill_real_time(group):
    # Add a segment ID that increments after each NA in `timestamp` 
    group['segment'] = group['timestamp'].notna().cumsum()
    group['t_pre'] = group.groupby('segment')['unix_arrival_time'].transform('first')

    # Group by 'segment' and get the first 'arrival_time' for each group
    first_arrival = group.groupby('segment')['unix_arrival_time'].first().shift(-1)
    # Map the shifted values back to the DataFrame
    group['t_next'] = group['segment'].map(first_arrival)
    # Fill NaN in the last segment with the last 'arrival_time'
    group['t_next'] = group['t_next'].fillna(group['unix_arrival_time'].iloc[-1])
    # Add columns for previous and next values
    group['r_pre'] = group['timestamp'].shift(1).ffill() 
    # Shift down for the previous value
    group['r_next'] = group['timestamp'].shift(-1).bfill()

    for i, row in group.iterrows():
        group.at[i, 'interpolated_time'] = interpolate_missing_times(row)
    return group

In [ ]:
# Apply the fill_real_time function group-wise
mixed_real_interpolated_data = complete_stop_time_data.groupby('trip_id', group_keys=False).apply(fill_real_time)

In [19]:
mixed_real_interpolated_data['final_arrival_time'] = mixed_real_interpolated_data['timestamp'].combine_first(\
    mixed_real_interpolated_data['interpolated_time'])

mixed_real_interpolated_data['final_departure_time'] = mixed_real_interpolated_data['timestamp'].combine_first(\
    mixed_real_interpolated_data['interpolated_time'])

# Add a source column to indicate which column the value came from
mixed_real_interpolated_data['interpolated'] = mixed_real_interpolated_data.apply(
    lambda row: '0' if pd.notna(row['timestamp']) else ('1' if pd.notna(row['interpolated_time']) else None), 
    axis=1
)

In [20]:
# Filter out any rows that don't have either a real or interpolated time (NA values).
mixed_real_interpolated_data = mixed_real_interpolated_data[\
    (mixed_real_interpolated_data.final_arrival_time.notna()) & \
    (mixed_real_interpolated_data.final_departure_time.notna())]

In [24]:
# Apply the function
tz = int(tz_offset(str(date), geo='Europe/London'))
mixed_real_interpolated_data['final_gtfs_arrival_time'] = unix_to_gtfs_time( mixed_real_interpolated_data['final_arrival_time'], date_with_dashes, tz=tz)
mixed_real_interpolated_data['final_gtfs_departure_time'] = unix_to_gtfs_time(mixed_real_interpolated_data['final_departure_time'], date_with_dashes, tz=tz)

In [26]:
# Set the 'arrival_time' and 'departure_time' as the final corrected time column.
mixed_real_interpolated_data['arrival_time'] = mixed_real_interpolated_data['final_gtfs_arrival_time']
mixed_real_interpolated_data['departure_time'] = mixed_real_interpolated_data['final_gtfs_departure_time']

In [27]:
realtime_timetable = mixed_real_interpolated_data.loc[:, ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
                                                          'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
                                                          'timepoint', 'route_id', 'interpolated']]

In [28]:
REAL_DIR = ROOT / f"data/gtfs/real-interpolated/{region}/{date}"
os.makedirs(os.path.abspath(REAL_DIR), exist_ok=True)

In [ ]:
RealRouteIDs = realtime_timetable['route_id']
RealRoutes = routes[routes['route_id'].isin(RealRouteIDs)]
RealRoutes.to_csv(REAL_DIR / "routes.txt", index=False)

In [ ]:
RealAgencyIDs = routes['agency_id']
RealAgencies = agencies[agencies['agency_id'].isin(RealAgencyIDs)]
RealAgencies.to_csv(REAL_DIR / "agency.txt", index=False)

In [ ]:
RealTripIDs = realtime_timetable['trip_id']
RealTrips = trips[trips['trip_id'].isin(RealTripIDs)]
RealTrips.to_csv(REAL_DIR / "trips.txt", index=False)

In [32]:
RealShapeIDs = RealTrips['shape_id'].unique()
RealShapes = shapes[shapes.shape_id.isin(RealShapeIDs)]
RealShapes.to_csv(REAL_DIR / "shapes.txt", index=False)

In [33]:
RealServiceIDs = RealTrips['service_id'].unique()
RealCalendar = calendar[calendar['service_id'].isin(RealServiceIDs)]
RealCalendar.to_csv(REAL_DIR / "calendar.txt", index=False)

In [ ]:
RealCalendarDates = calendar_dates[calendar_dates['service_id'].isin(RealServiceIDs)]
RealCalendarDates.to_csv(REAL_DIR / "calendar_dates.txt", index=False)

In [34]:
# Filtering only columns we need to write stop_times.txt
RealStopTimes = realtime_timetable.loc[:, ['trip_id','arrival_time','departure_time','stop_id','stop_sequence','stop_headsign','pickup_type','drop_off_type','shape_dist_traveled','timepoint','interpolated']]
RealStopTimes.to_csv(REAL_DIR / "stop_times.txt",index=False)

In [35]:
RealStopIDs = realtime_timetable['stop_id'].to_list()
RealStops = stops[stops.stop_id.isin(RealStopIDs)].copy()

# Location type must be able to be parsed as an integer.
RealStops['location_type'] = RealStops['location_type'].astype('Int64')
RealStops.drop(columns='parent_station', inplace=True)
RealStops.to_csv(REAL_DIR / "stops.txt", index=False)

In [36]:
# Writing feed_info.txt
feed_info.to_csv(REAL_DIR / "feed_info.txt", index=False)

In [ ]:
# Zip the GTFS timetables into a single GTFS.zip file.
zip_directory(REAL_DIR, ROOT / 'data/real-interpolated', f'{region}_{date}.gtfs')